In [1]:
import os

In [2]:
%pwd

'd:\\Mlops_Projects\\DS_Project1\\research'

In [3]:
os.chdir('D:/Mlops_Projects/DS_Project1/')

In [4]:
%pwd

'D:\\Mlops_Projects\\DS_Project1'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.datascience_project.constants import *
from src.datascience_project.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # read data_ingestion section from config (self.config is a dict)
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [9]:
import os
import urllib.request as request
from src.datascience_project import logger
import zipfile

In [11]:
##component-Data_ingestion  
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    ##Download the zip file
    def download_file(self):
        # logic to download data from self.config.source_url
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with info: \n{headers}")
        else:
            logger.info(f"File already exists")
    
    ##Unzip the downloaded file
    def extract_zip_file(self):
        # logic to unzip the file to self.config.unzip_dir
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"File extracted to {unzip_path}")
    


In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)

2025-11-06 05:47:07,566: INFO: common: YAML file: config\config.yaml loaded successfully]
2025-11-06 05:47:07,570: INFO: common: YAML file: params.yaml loaded successfully]
2025-11-06 05:47:07,571: INFO: common: YAML file: schema.yaml loaded successfully]
2025-11-06 05:47:07,574: INFO: common: Directory created at: artifacts]
2025-11-06 05:47:07,576: INFO: common: Directory created at: artifacts/data_ingestion]
2025-11-06 05:47:08,594: INFO: 1802960603: artifacts/data_ingestion/data.zip downloaded with info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C3D3:31FC60:1483F:3D769:690BE903
Accept-Ranges: bytes
Date: Thu, 06 Nov 2025 00:17:0